In [9]:
import pandas as pd
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
import re

In [2]:
data_oracle = pd.read_excel("C:\\Users\\Lenovo\\Desktop\\Final File.xlsx", sheet_name='Oracle Data')

In [3]:
# Count the number of unique products in the ORDERED_ITEM column
unique_products_count = data_oracle['ORDERED_ITEM'].nunique()

print(f'The number of unique products is: {unique_products_count}')

The number of unique products is: 640


In [8]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time

# Assuming your unique product ids are stored in a list
unique_product_ids = data_oracle['ORDERED_ITEM'].unique().tolist()

# Remove the specific ID 'SHIPPING_LOCAL' from the list
if 'SHIPPING_LOCAL' in unique_product_ids:
    unique_product_ids.remove('SHIPPING_LOCAL')

# Create a DataFrame to store the scraped data
product_data = pd.DataFrame(columns=['Product ID', 'Product Name', 'Category'])

# Setup Selenium WebDriver
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Base URL of the e-commerce website search page
base_url = 'https://shop.cosmaline.com/website/search?search='

# Process only the first 5 product IDs for testing
for product_id in unique_product_ids:
    try:
        # Construct the search URL
        search_url = f'{base_url}{product_id}&order=name+asc'
        
        # Fetch the search results page
        driver.get(search_url)
        
        # Allow the page to load
        time.sleep(3)  # Adjust the sleep time if necessary
        
        # Parse the page content
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Extract product name from the search results
        product_name_tag = soup.find('a', {'class': 'dropdown-item p-2 text-wrap'})
        if product_name_tag:
            product_name = product_name_tag.get('href').split('/')[-1].replace('-', ' ').split()[:-2]
            product_name = ' '.join(product_name)
        else:
            product_name = "Product Not Found"
        
        # Extract categories from the search results
        category_tags = soup.find_all('button', {'class': 'btn btn-link btn-sm p-0'})
        categories = [tag.text.strip() for tag in category_tags]  # Get all category buttons
        
        # Append the data to the DataFrame
        new_row = pd.DataFrame({'Product ID': [product_id], 'Product Name': [product_name], 'Category': [categories]})
        product_data = pd.concat([product_data, new_row], ignore_index=True)
    
    except Exception as e:
        print(f'Error scraping product ID {product_id}: {e}')
        new_row = pd.DataFrame({'Product ID': [product_id], 'Product Name': ["Error Occurred"], 'Category': [[]]})
        product_data = pd.concat([product_data, new_row], ignore_index=True)

# Close the WebDriver
driver.quit()

# Save the scraped data to a CSV file
product_data.to_csv('C:\\Users\\Lenovo\\Desktop\\scraped_product_data_with_categories.csv', index=False)

# Display the DataFrame
print(product_data)

    Product ID                                       Product Name  \
0     C0003751  c0003751 eimi thermal image heat protection spray   
1     C0004198   c0004198 invigo nutri enrich frizz control cream   
2     C0003733                               c0003733 eimi dry me   
3       174347        cosmaline hair and body mist floral fantasy   
4       174750                     soft wave hair lightener spray   
..         ...                                                ...   
634   D0009899                    catrice 48h power stay brow gel   
635     196573              cosmal cure professional sulfate free   
636     193850              cosmal cure professional sulfate free   
637     196450              cosmal cure professional sulfate free   
638     197912            cosmaline eco deodorant soft confidence   

                                              Category  
0                           [Hair Styling, Hair Spray]  
1    [Hair Care, Frizzy or Dry & Damaged Hair, Friz...  


In [17]:
product_data = pd.read_csv('C:\\Users\\Lenovo\\Desktop\\scraped_product_data_with_categories.csv')

# Function to clean the product name by removing the ID if it appears at the start
def clean_product_name(row):
    product_id = str(row['Product ID']).lower()
    product_name = str(row['Product Name'])
    if product_name.lower().startswith(product_id):
        product_name = product_name[len(product_id):].strip()
    return product_name

# Apply the cleaning function to the DataFrame
product_data['Product Name'] = product_data.apply(clean_product_name, axis=1)

# Save the cleaned data to a new CSV file
product_data.to_csv('C:\\Users\\Lenovo\\Desktop\\cleaned_product_data.csv', index=False)

# Display the cleaned DataFrame
print(product_data)

    Product ID                                 Product Name  \
0     C0003751     eimi thermal image heat protection spray   
1     C0004198      invigo nutri enrich frizz control cream   
2     C0003733                                  eimi dry me   
3       174347  cosmaline hair and body mist floral fantasy   
4       174750               soft wave hair lightener spray   
..         ...                                          ...   
634   D0009899              catrice 48h power stay brow gel   
635     196573        cosmal cure professional sulfate free   
636     193850        cosmal cure professional sulfate free   
637     196450        cosmal cure professional sulfate free   
638     197912      cosmaline eco deodorant soft confidence   

                                              Category  
0                       ['Hair Styling', 'Hair Spray']  
1    ['Hair Care', 'Frizzy or Dry & Damaged Hair', ...  
2                         ['Hair Care', 'Dry Shampoo']  
3              